<a href="https://colab.research.google.com/github/liangyao910/ColabNotebooks/blob/master/chatgpt_api_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
! pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 8.6 MB/s eta 0:00:00


In [2]:
! pip3 install SpeechRecognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 25.1 MB/s eta 0:00:00


In [3]:
!pip install gtts
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993243 sha256=41982e59cc82aec31d8fdba53611fe9370f314fd3c4c0c63fe3e208f69c84f06
  Stored in directory: /root/.cache/pip/wheels/d1/c1/d9/7e068de779d863bc8f8fc9467d85e25cfe47fa5051fff1a1bb
Successfully built langdetect


# 音声入力

In [18]:
from IPython.display import Javascript
import IPython.display
from google.colab import output
from base64 import b64decode
import librosa
# import librosa.display
import soundfile as sf
import speech_recognition as sr

RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec, filename='audio.wav'):
  display(Javascript(RECORD))
  s = output.eval_js('record(%d)' % (sec * 1000))
  b = b64decode(s.split(',')[1])
  with open(filename, 'wb+') as f:
    f.write(b)

# 音声ファイルからテキストに変換する
def recognize_speech(file_path, UESER_LANG):
  r = sr.Recognizer()
  with sr.AudioFile(file_path) as source:
      audio = r.record(source) # 音声ファイルを読み込む
  try:
      if UESER_LANG == "日本語":
        text = r.recognize_google(audio, language='ja-JP') # 音声をテキストに変換する
      if UESER_LANG == "中国語":
        text = r.recognize_google(audio, language='zh-TW') # 音声をテキストに変換する
      if UESER_LANG == "英語":
        text = r.recognize_google(audio, language='en-US') # 音声をテキストに変換する
      return text
  except sr.UnknownValueError:
      print("Google Speech Recognition could not understand audio")
  except sr.RequestError as e:
      print("Could not request results from Google Speech Recognition service; {0}".format(e))
  return

def audio_to_text(UESER_LANG):
  # 録音
  audiofile = "audio.wav"
  second = 5
  print(f"Speak to your microphone {second} sec...")
  record(second, audiofile)
  print("Done!")
  
  speech, rate = librosa.load(audiofile, sr=16000, mono=False)
  sf.write('output.wav', speech, rate, subtype="PCM_16")

  # 音声ファイルのパス
  file_path = "output.wav"

  # 音声をテキストに変換する
  text = recognize_speech(file_path, UESER_LANG)
  # "的英语日语的口语说法是"
  # "有没有其他说法"

  # 波形プロットにより録音を確認
  # librosa.display.waveshow(speech, sr=rate)

  return speech, rate, text

  # 入力言語選択
UESER_LANG = "\u4E2D\u56FD\u8A9E" #@param ["中国語", "日本語", "英語"] {type:"string"}
print('-----------------------------------------------------------------------------------------\n')
print('入力言語:',UESER_LANG)
print('\n-----------------------------------------------------------------------------------------')

speech, rate, text = audio_to_text(UESER_LANG)

# 変換したテキストを表示する
print('****************-------------input_text--------------------****************\n')
print(text)
print('\n-----------------------------------------------------------------------------------------')
# 録音を確認
display(IPython.display.Audio(data=speech, rate=rate))

-----------------------------------------------------------------------------------------

入力言語: 中国語

-----------------------------------------------------------------------------------------
Speak to your microphone 5 sec...


<IPython.core.display.Javascript object>

Done!


<ipython-input-18-e41fbac10fd2>:67: UserWarning: PySoundFile failed. Trying audioread instead.
  speech, rate = librosa.load(audiofile, sr=16000, mono=False)
/usr/local/lib/python3.9/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


****************-------------input_text--------------------****************

刚跑完步脚好酸的日文根英文怎么说

-----------------------------------------------------------------------------------------


# Chatgpt

In [19]:
import openai
openai.api_key = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

res = openai.ChatCompletion.create(
     model="gpt-3.5-turbo",
          messages=[
          {
              "role": "system",
              "content": f"请用{UESER_LANG}回答。"
          },
          {
              "role": "user",
              "content": text
          },
     ],
 )

# print(res)
response_text = res["choices"][0]["message"]["content"]
print(response_text)

# jsonで履歴保存
import pandas as pd
import json
import datetime
import os 

dt_now = dt_now_jst_aware = datetime.datetime.now(
    datetime.timezone(datetime.timedelta(hours=9))
)
time = dt_now.strftime('%Y-%m-%d %H:%M:%S')
path = '/content/drive/MyDrive/Colab Notebooks/json/chatgpt.json'
df = pd.DataFrame({'time': [time], 'ask': [text], 'response': [response_text]})
if os.path.exists(path):
  df_read = pd.read_json(path)
  df_output = pd.concat([df_read, df], ignore_index = True)
else:
  df_output = df

df_output.to_json(path, force_ascii=False)
display(df_output.head())

日语：
今ランニングを終えたばかりで、足がとても痛いです。

英语：
I just finished running and my feet are very sore.


,time,ask,response
0,2023-03-27 14:26:45,これは何ですか の英語と中国語を翻訳してください,英語：What is this?\n日本語：これは何ですか？\n\n中国語：这是什么？
1,2023-03-27 14:28:42,卫星地图的日文跟英文翻译,卫星地图的日文是「衛星写真」(えいせいしゃしん)。\n\n卫星地图的英文是「satellit...
2,2023-03-27 23:32:12,现在不早了的英文翻译跟日文翻译是,英文翻譯：It's not early now.\n\n日文翻譯：もう早くないです。
3,2023-03-27 23:41:30,刚跑完步脚好酸的日文根英文怎么说,日语：\n今ランニングを終えたばかりで、足がとても痛いです。\n\n英语：\nI just ...


# テキスト読み上げ

In [20]:
from gtts import gTTS
import librosa
import os
import IPython.display as ipd
from langdetect import detect_langs
import re
import time

# Colab読み上げ用波形データ変換
def get_audio_wave(filename):
  # 音の波形を取得
  y, sr = librosa.load(filename)
  return y, sr

# テキスト加工（{言語：読み上げ内容}）
# ・文ごとに切り分け(-:,.。"\'・()（）、) 
sentences = re.split('[-:,.。"\'・()（）、：\n]', response_text)

# ・言語分類
def detect_lang(text):
  langs = detect_langs(text)
  for lang in langs:
      print('------------------------------------')
      print(f"Language: {lang.lang}, Probability: {lang.prob}")
      break
  return langs[0].lang

# 音声作成、テキストが多い場合、自動読み上げできません
# テキスト変換
def save_audio_from_text(text, aduio_path, lang):
  tts = gTTS(text=text, lang=lang)
  if 'ko' in lang:
    lang = 'zh'
  if 'ro' in lang:
    lang = 'en'
  if 'id' in lang:
    lang = 'en'
  if 'hr' in lang:
    lang = 'en'
  tts = gTTS(text=text, lang=lang)
  tts.save(aduio_path)

def read_text(text):
  file_name = "read_text.mp3"
  lang = detect_lang(text)
  save_audio_from_text(text, file_name, lang)
  y, sr = get_audio_wave(file_name)
  if os.path.exists(file_name):
    os.remove(file_name)
  return y, sr

# 読み上げ
for sentence in sentences:
  wait_sec = 0
  try:
    y, sr = read_text(sentence)
    wait_sec = int(len(y)/sr) + 0.5
    print('------------------------------------\n')
    print(sentence)
    print('\n------------------------------------')
  except:
    situation = 'no text'
    
  if wait_sec > 0:
    display(ipd.Audio(y, rate=sr, autoplay=True))
    time.sleep(wait_sec)

------------------------------------
Language: zh-cn, Probability: 0.9999967252776865
------------------------------------

日语

------------------------------------


------------------------------------
Language: ja, Probability: 0.9999999999959939
------------------------------------

今ランニングを終えたばかりで

------------------------------------


------------------------------------
Language: ja, Probability: 0.9999987674030902
------------------------------------

足がとても痛いです

------------------------------------


------------------------------------
Language: ko, Probability: 0.9999986972924503
------------------------------------

英语

------------------------------------


------------------------------------
Language: en, Probability: 0.7142833501634523
------------------------------------

I just finished running and my feet are very sore

------------------------------------
